In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define paths
train_dir = 'Train'
validation_dir = 'Validation'

# Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply shearing
    zoom_range=0.2,  # Apply zoom
    horizontal_flip=True  # Randomly flip images horizontally
)

# Train Generator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical'  # Use categorical for multi-class classification
)

# Validation Generator
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting
    Dense(len(train_generator.class_indices), activation='softmax')  # Output Layer
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=2 # Adjust as needed
)

# Evaluate Model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

import numpy as np
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)


Found 15447 images belonging to 42 classes.
Found 3171 images belonging to 42 classes.


C:\Users\Gopinadh\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Gopinadh\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
289/482 ━━━━━━━━━━━━━━━━━━━━ 4:16 1s/step - accuracy: 0.2291 - loss: 2.9850

C:\Users\Gopinadh\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


482/482 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2706 - loss: 2.8234

C:\Users\Gopinadh\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\Gopinadh\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


482/482 ━━━━━━━━━━━━━━━━━━━━ 946s 2s/step - accuracy: 0.2707 - loss: 2.8227 - val_accuracy: 0.0470 - val_loss: 4.2271
Epoch 2/2
  1/482 ━━━━━━━━━━━━━━━━━━━━ 8:37 1s/step - accuracy: 0.5000 - loss: 2.1049

C:\Users\Gopinadh\anaconda3\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


482/482 ━━━━━━━━━━━━━━━━━━━━ 212s 438ms/step - accuracy: 0.5000 - loss: 2.1049 - val_accuracy: 0.0483 - val_loss: 4.1640
100/100 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.0487 - loss: 4.1757
Validation Loss: 4.160606384277344, Validation Accuracy: 0.04982655495405197
100/100 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step


In [9]:
from flask import Flask, request, render_template
import os

app = Flask(__name__)

# Set the upload folder
UPLOAD_FOLDER = 'test'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def upload_form():
    return render_template("templates\hktnm1.html")  # Render the HTML form

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'image' not in request.files:
        return "No file part"
    
    file = request.files['image']
    if file.filename == '':
        return "No selected file"
    
    if file:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filepath)  # Save the file
        return f"File {file.filename} uploaded successfully!"
    

    
predicted_class = np.argmax(predictions, axis=1)

# Map predicted class to disease name (use class indices from train_generator)
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}  # Reverse the class indices
predicted_disease = class_labels[predicted_class[0]]

print(f'The plant is predicted to have: {predicted_disease}')

The plant is predicted to have: Becterial Blight in Rice
